# Ensemble Learning

## Initial Imports

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install -U imbalanced-learn

  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install -U imbalanced-learn


  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)


In [40]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [41]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [42]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [43]:
# Create our features
 # YOUR CODE HERE
X = df.drop(columns='loan_status')
#X = pd.get_dummies(df.drop(columns='loan_status', axis=1))

#X.head()

# Create our target
 # YOUR CODE HERE
y = df['loan_status']

In [44]:
#y = df['loan_status']

#y.head()

In [45]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [46]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [47]:
cat_cols = list(X.select_dtypes(include=object).columns)

In [48]:
X=pd.get_dummies(X,columns=cat_cols,drop_first=True)
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,1,1,0,0,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,0,0,1,0,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,0,0,1,0,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,0,1,0,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,0,0,0,1,1,1,0


In [49]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape



#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

(51612, 86)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [50]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [51]:
from sklearn.preprocessing import StandardScaler
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

#X_scaler = scaler.fit(X_train)
#scaler.fit(X_train)
#X_scaler = scaler.fit(X_train)
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)




In [52]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#y_train_scaled = y_scaler.transform(y_train)
#y_test_scaled = y_scaler.transform(y_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [56]:
from imblearn.ensemble import BalancedRandomForestClassifier
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE

brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [57]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
#brf_predictions = brf.predict(X_test)
y_pred = brf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7952749878127883

In [58]:
# Display the confusion matrix
# YOUR CODE HERE
#confusion_matrix(y_test, predictions)
confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 2090, 15028]], dtype=int64)

In [59]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
#print("Imbalanced Classification Report")
#print(classification_report_imbalanced(y_test, brf_predictions))
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.71      0.88      0.06      0.79      0.62        87
   low_risk       1.00      0.88      0.71      0.93      0.79      0.64     17118

avg / total       0.99      0.88      0.71      0.93      0.79      0.64     17205



In [66]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
#importances = brf_model.feature_importances_
#importances_sorted = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
#importances_sorted[:10]
pd.set_option("display.max_rows", None, "display.max_columns", None)

importances = pd.DataFrame(brf_model.feature_importances_, index = X_train.columns, columns=['Importance']).sort_values('Importance', ascending=False)
importances.loc[:,:]



,Importance
total_rec_prncp,0.078795
total_rec_int,0.068296
total_pymnt_inv,0.063323
total_pymnt,0.060703
last_pymnt_amnt,0.048325
int_rate,0.030010
mths_since_recent_inq,0.019435
dti,0.019336
installment,0.018303
issue_d_Jan-2019,0.018107


### Easy Ensemble Classifier

In [67]:
from imblearn.ensemble import EasyEnsembleClassifier 
# Train the Classifier
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=1)
eec = EasyEnsembleClassifier(n_estimators=100,random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [75]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
#y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.9317770031212663

In [82]:
# Display the confusion matrix
# YOUR CODE HERE
#print(confusion_matrix(y_test, y_pred))
#confusion_matrix(y_test, y_pred)
cm_eec = confusion_matrix(y_test, y_pred)
cm_df_eec = pd.DataFrame(
    cm_eec, index=["The High Risk", "The Low Risk"], columns=["Predict of High Risk", "Predict of Low Risk"]
)
cm_df_eec

,Predict of High Risk,Predict of Low Risk
The High Risk,93,8
The Low Risk,979,16125


In [65]:
# Print the imbalanced classification report
# YOUR CODE HERE
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE. 
    Best balance accuracy score the Easy Esemble Classifier.

2. Which model had the best recall score?

    YOUR ANSWER HERE.
    Best recall score is the Easy Esemble Classifier

    ***

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
    Best best geometric mean score is the Easy Ensemble model. 

4. What are the top three features?

    YOUR ANSWER HERE.
    Total Rec Prncp
    Total Payment Invoice    
    Last Payment Amount


    